In [1]:
!pip install pyspark >> None

# Задание

Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.

Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [10]:
# Создание сессии Spark
spark = SparkSession.builder \
    .appName("Average Revenue by Category") \
    .getOrCreate()

# Пример данных
data = [("2023-11-20", "Electronics", 100, 12000),
        ("2023-11-21", "Electronics", 110, 13000),
        ("2023-11-22", "Electronics", 105, 12500),
        ("2023-11-20", "Clothing", 300, 15000),
        ("2023-11-21", "Clothing", 280, 14000),
        ("2023-11-22", "Clothing", 320, 16000),
        ("2023-11-20", "Books", 150, 9000),
        ("2023-11-21", "Books", 200, 12000),
        ("2023-11-22", "Books", 180, 10000)]

# Создание DataFrame
df = spark.createDataFrame(data, ["date", "category", "quantity", "revenue"])

df.show()

+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [12]:
# Оконная функция для рассчета средней выручки по категориям продуктов
window_spec = Window.partitionBy("category")

# Добавление столбца со средней выручкой
df_avg_revenue = df.withColumn("avg_revenue", F.round(avg(col("revenue")).over(window_spec), 2))

df_avg_revenue.show()

+----------+-----------+--------+-------+-----------+
|      date|   category|quantity|revenue|avg_revenue|
+----------+-----------+--------+-------+-----------+
|2023-11-20|      Books|     150|   9000|   10333.33|
|2023-11-21|      Books|     200|  12000|   10333.33|
|2023-11-22|      Books|     180|  10000|   10333.33|
|2023-11-20|   Clothing|     300|  15000|    15000.0|
|2023-11-21|   Clothing|     280|  14000|    15000.0|
|2023-11-22|   Clothing|     320|  16000|    15000.0|
|2023-11-20|Electronics|     100|  12000|    12500.0|
|2023-11-21|Electronics|     110|  13000|    12500.0|
|2023-11-22|Electronics|     105|  12500|    12500.0|
+----------+-----------+--------+-------+-----------+



In [13]:
# Операция pivot для преобразования данных
pivot_df = df_avg_revenue.groupBy("category").pivot("date").avg("avg_revenue")

# Отображение результата
pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|      Books|  10333.33|  10333.33|  10333.33|
|   Clothing|   15000.0|   15000.0|   15000.0|
|Electronics|   12500.0|   12500.0|   12500.0|
+-----------+----------+----------+----------+



In [14]:
# или

# Операция pivot для преобразования данных
pivot_df_2 = df_avg_revenue.groupBy("category").pivot("date").mean("revenue")

# Отображение результата
pivot_df_2.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+

